### 1. softmax_classifier

모델 테스트 시 backend.eval()과 np.argmax() 아무거나 써도 상관 없음

backend.eval()

- TensorFlow에서 제공하는 함수
- TensorFlow 연산을 평가하여 파이썬 값으로 변환
- 가장 큰 값의 인덱스를 TensorFlow 연산으로 계산하고 평가


np.argmax()

- np.argmax()는 NumPy에서 제공하는 함수
- 배열에서 가장 큰 값의 인덱스를 NumPy 배열 연산으로 계산
- NumPy 배열의 차원을 줄이는 용도로 주로 사용


In [3]:
import tensorflow as tf
import numpy as np

x_raw = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_raw = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_data = np.array(x_raw, dtype=np.float32)
y_data = np.array(y_raw, dtype=np.float32)

nb_classes = 3 #one hot encoding: 3개로 분류

tf.model = tf.keras.Sequential() #모델 초기화
tf.model.add(tf.keras.layers.Dense(input_dim=4, units=nb_classes, use_bias=True))  # hypothesis: 연결 레이어 추가: input 4, output 3
# use softmax activations: softmax = exp(logits) / reduce_sum(exp(logits), dim)
tf.model.add(tf.keras.layers.Activation('softmax')) #softmax 함수로 확률로 변환

# use loss == categorical_crossentropy
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.1), metrics=['accuracy'])
tf.model.summary()

history = tf.model.fit(x_data, y_data, epochs=2000) # 모델 학습


#argmax로 모델 테스트
print('--------------')
# Testing & One-hot encoding
a = tf.model.predict(np.array([[1, 11, 7, 9]]))
print(a, tf.keras.backend.eval(tf.argmax(a, axis=1))) # 축1에서 가장 큰 값 반환

print('--------------')
b = tf.model.predict(np.array([[1, 3, 4, 3]]))
print(b, tf.keras.backend.eval(tf.argmax(b, axis=1)))

print('--------------')
# or use argmax embedded method, predict_classes
c = tf.model.predict(np.array([[1, 1, 0, 1]]))
c_onehot = np.argmax(c, axis=-1)
print(c, c_onehot)

print('--------------')
all = tf.model.predict(np.array([[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]])) #3개 한꺼번에 질문
all_onehot = np.argmax(all, axis=-1)
print(all, all_onehot)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 3)                 15        


                                                                 
 activation_2 (Activation)   (None, 3)                 0         
                                                                 
Total params: 15 (60.00 Byte)
Trainable params: 15 (60.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/2000
1/1 [==============================] - 0s 192ms/step - loss: 4.9681 - accuracy: 0.3750
Epoch 2/2000
1/1 [==============================] - 0s 2ms/step - loss: 4.7127 - accuracy: 0.3750
Epoch 3/2000
1/1 [==============================] - 0s 3ms/step - loss: 4.4661 - accuracy: 0.3750
Epoch 4/2000
1/1 [==============================] - 0s 2ms/step - loss: 4.2303 - accuracy: 0.3750
Epoch 5/2000
1/1 [==============================] - 0s 3ms/step - loss: 4.0075 - accuracy: 0.3750
Epoch 6/2000
1/1 [==============================] - 0s 2ms/step - loss: 3.7995 - accuracy: 0.3750
Epoch 7/2000
1/1 [============================

### 2. softmax_zoo_classifier

In [6]:
import tensorflow as tf
import numpy as np

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7  # 0 ~ 6

Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)  # one hot encoding
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # reshape

W = tf.Variable(tf.random.normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

def cost_func():
    logits = hypothesis(x_data)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot))
    return cost

def grad_func():
    with tf.GradientTape() as tape:
        cost = cost_func()
        grads = tape.gradient(cost, [W, b])
        return grads

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# Training
for step in range(2001):
    grads = grad_func()
    optimizer.apply_gradients(zip(grads, [W, b]))
    if step % 100 == 0:
        print("Step: {:5}\tCost: {:.3f}".format(step, cost_func()))

# Test
def accuracy_func():
    logits = hypothesis(x_data)
    prediction = tf.argmax(logits, 1)
    correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

print("Accuracy: ", accuracy_func().numpy())

# Let's see if we can predict
logits = hypothesis(x_data)
prediction = tf.argmax(logits, 1)

for p, y in zip(prediction.numpy(), y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


Step:     0	Cost: 2.023
Step:   100	Cost: 1.683
Step:   200	Cost: 1.665
Step:   300	Cost: 1.546
Step:   400	Cost: 1.508
Step:   500	Cost: 1.493
Step:   600	Cost: 1.483
Step:   700	Cost: 1.475
Step:   800	Cost: 1.468
Step:   900	Cost: 1.462
Step:  1000	Cost: 1.457
Step:  1100	Cost: 1.453
Step:  1200	Cost: 1.450
Step:  1300	Cost: 1.447
Step:  1400	Cost: 1.445
Step:  1500	Cost: 1.443
Step:  1600	Cost: 1.441
Step:  1700	Cost: 1.440
Step:  1800	Cost: 1.438
Step:  1900	Cost: 1.437
Step:  2000	Cost: 1.436
Accuracy:  0.7326733
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 2 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[False] Prediction: 2 True Y: 3
[False] Prediction: 2 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[False] Prediction: 2 True Y: 3
[False] Prediction: 2 True Y: 6
[True] Prediction: 6 True Y: 6
[True

In [4]:
# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

'''
(101, 16) (101, 1)
'''

nb_classes = 7  # 0 ~ 6

# Convert y_data to one_hot
y_one_hot = tf.keras.utils.to_categorical(y_data, nb_classes)
print("one_hot:", y_one_hot)

tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.Dense(units=nb_classes, input_dim=16, activation='softmax'))
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.1), metrics=['accuracy'])
tf.model.summary()

history = tf.model.fit(x_data, y_one_hot, epochs=1000)

# Single data test
test_data = np.array([[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0]]) # expected prediction == 3 (feathers)
print(tf.model.predict(test_data), np.argmax(tf.model.predict(test_data), axis=-1)) # 예측에 대해 가장 큰 값

# Full x_data test
pred = np.argmax(tf.model.predict(x_data), axis=-1)
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot: [[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0